# Start to finish
rough draft to implement in pycharm project

In [14]:
import librosa
import numpy as np
import torch
from pathlib import Path
import torch.nn.functional as F
import IPython

## Select tap sound

In [15]:
# Audio extracted from youtube
clip = '../../../../Documents/Source/Shuffle/4/2.wav'
clip_path = Path(clip)
y, sr = librosa.load(clip)

In [16]:
# Play sound
IPython.display.Audio(data=y, rate=sr)

## Processing Pipeline

In [21]:
# Parameters (set in training and validation)
clip_length = 20772
n_mfcc = 20
frame_length = 256
hop_length = 128

In [22]:
def resize_signal(path, length):
    samples, sample_rate = librosa.load(path)
    if len(samples) < length:
        y = np.pad(samples, (0, length-len(samples)), 'constant')
    elif len(samples) > length:
        y = samples[:length]
    else:
        y = samples
    return y, sample_rate

def get_features_mfcc(samples, sample_rate):
    mfccs = np.mean(librosa.feature.mfcc(y=samples, sr=sample_rate, n_mfcc=n_mfcc).T,axis=0)
    return mfccs

def get_features_zcr(samples, sample_rate):
    zcr = librosa.feature.zero_crossing_rate(samples, frame_length=frame_length, hop_length=hop_length)
    return zcr
    
def get_features_energy(samples, sample_rate) :
    energy = np.array([sum(abs(samples[i:i+frame_length]**2)) for i in range (0, len(samples), hop_length)])
    return energy

def get_features_rmse(samples, sample_rate):
    rmse = librosa.feature.rmse(samples, frame_length=frame_length, hop_length=hop_length, center=True)
    return rmse[0]

def get_features_bpm(samples, sample_rate): 
    onset_env = librosa.onset.onset_strength(samples, sr=sample_rate) # Assumes static tempo, for dynamic: aggregate=None
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sample_rate)
    return tempo

def get_label(path):
    if path.parts[-3] == 'Shuffle':
        return 1
    else:
        return 0

In [23]:
# Pad with ending silence or cut to set length
y, sr = resize_signal(clip_path, clip_length)


In [24]:
# Extract features
mfcc = get_features_mfcc (y, sr)[:,np.newaxis]
zcr = get_features_zcr (y, sr).T
energy = get_features_energy(y, sr)[:,np.newaxis]
rmse = get_features_rmse (y, sr)[:,np.newaxis]
bpm = get_features_bpm (y, sr)[:,np.newaxis]


## Select features

In [34]:
feature_set = (mfcc)


In [35]:
# Combine features to form input for model
inputs = np.concatenate(feature_set, axis=0).T

## Select model

In [36]:
model = ('../../../../Desktop/TapDanceDecoder_LOCAL_BACKUP/src/train/trained_models/one_hidden_mfcc_128.pt')


## Run and predict

In [37]:
dtype = torch.float
device = torch.device('cpu')

inputs = torch.tensor(inputs, device=device, dtype=dtype)

# Load model
model = torch.load(model)

outputs = model(inputs)
#print(outputs)
#print((F.softmax(outputs)))

y_pred = (torch.argmax(outputs.data).numpy())

true = get_label(clip_path)


if y_pred == 1:
    print('Predicted: Shuffle')
elif y_pred == 0:
    print('Predicted: Ball change')
print()
if true == 1:
    print('It was a Shuffle.')
elif true == 0:
    print('It was a Ball change.')


Predicted: Ball change

It was a Shuffle.


In [38]:
# What did that sound like again?
IPython.display.Audio(data=y, rate=sr)